<div style="text-align: center; padding: 30px">
  <h1><strong>Actividades con Conflicto</strong></h1>
  <h3><strong>Ejercicio 0</strong></h3>
</div>

El departamento de una universidad debe hacer la programación de los exámenes finales de los cursos que imparte dicho departamento. Cada examen tiene una duración de dos horas. Se necesita determinar el número de días necesario para la realización de dichos exámenes en los siguientes periodos de tiempo: 8:00 a 10:00, 10:15 a 12:15, 14:00 a 16:00 y 16:15 a 18:15, que dan como resultado cuatro periodos de dos horas por día, que deben servir para programar los exámenes de los cursos en distintos salones de clase. Para cada uno de los cursos, se proporciona una lista de los cursos que son incompatibles (ver Tabla 1), es decir, cursos que no se pueden programar al mismo tiempo, porque ambos están siendo por cursados por uno o varios de los estudiantes. Se requiere formular el problema para determinar el número mínimo de días necesarios para la realización de los exámenes de los 11 cursos. 

In [60]:
from ortools.linear_solver import pywraplp

### **Uso de Python y el Solver SCIP**



En Python, utilizamos el solver `SCIP` para definir y resolver este problema. Para declarar variables binarias, se utiliza el método:

```python
solver.BoolVar(name)
```

In [61]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjuntos de Índices**

- **C**: Conjunto de cursos, $C = \{1, 2, \dots, 11\}$
- **P**: Conjunto de periodos, $P = \{1, 2, \dots, 11\}$
- **I**: Conjunto de pares $(i, j)$ tales que $i, j \in C$ y están en conflicto.
- Para cada $i \in C$, **$I_i$**: Conjunto de cursos incompatibles con $i$.  
    Ejemplo: $I_1 = \{2, 5, 7, 10, 11\}$

In [62]:
# Conjuntos
C = set(range(1, 12))  # Cursos {1, 2, ..., 11}
P = set(range(1, 5))   # Periodos {1, 2, 3, 4}
D = set(range(1, 12))  # Días (máximo 11 días posibles)

# Incompatibilidades (ejemplo: curso 1 es incompatible con cursos 2, 5, 7, 10 y 11)
I = [(1, 2), (1, 5), (1, 7), (1, 10), (1, 11),
     (2, 1), (2, 5), (2, 7), (2, 10), (2, 11),
     (4, 3), (4, 5), (4, 6), (4, 7), (4, 10), (4, 11),
     (3, 4), (3, 5), (3, 6), (3, 7), (3, 9), (3, 10), (3, 11),
     (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11),
     (6, 3), (6, 4), (6, 5), (6, 7), (6, 9), (6, 10), (6, 11),
     (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (7, 11),
     (8, 5), (8, 7), (8, 10), (8, 11),
     (9, 3), (9, 5), (9, 6), (9, 7), (9, 10), (9, 11),
     (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 11),
     (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10) 
     ]

### **Parámetros**



Los parámetros capturan la información del problema, que en este caso se refiere a los cursos en conflicto. Esto puede representarse de la siguiente manera:

```math
\forall i, j \in C: 
a_{i,j} = 
\begin{cases} 
1 & \text{si } i \text{ y } j \text{ están en conflicto}, \\ 
0 & \text{en otro caso}.
\end{cases}
```

### **Variables de Decisión**



Las variables de decisión están relacionadas con la respuesta al problema: determinar en qué periodo se programa el examen de cada curso. 

- Para cada $i \in C$ y $j \in P$: 
    ```math
    x_{i,j} = 
    \begin{cases} 
    1 & \text{si el examen del curso } i \text{ se asigna al periodo } j, \\ 
    0 & \text{en otro caso}.
    \end{cases}
    ```

- Sea **$y$**: el número mínimo de días necesarios para programar los exámenes.

- Para cada $k \in P$: 
    ```math
    z_k = 
    \begin{cases} 
    1 & \text{si se utiliza el periodo } k, \\ 
    0 & \text{en otro caso}.
    \end{cases}
    ```



$$
\begin{aligned}
x_{i,j} &= 
\begin{cases} 
1 & \text{si el examen del curso $i$ se asigna al periodo $j; \quad \forall i \in C, \forall j \in P$} \\ 
0 & \text{en otro caso}.
\end{cases} \\

y &= \text{ el número mínimo de días necesarios para programar los exámenes. }
\\

z_k &= 
\begin{cases} 
1 & \text{si se utiliza el periodo $k; \quad \forall k \in P$} \\ 
0 & \text{en otro caso}.
\end{cases}

\end{aligned} 
$$


In [63]:
# Variables de decisión
x = {}  # x[i, d, p] = 1 si el curso i se programa el día d en el periodo p
for i in C: # Cursos
    for d in D: # Días
        for p in P: # Periodos
            x[i, d, p] = solver.BoolVar(f'x[{i},{d},{p}]') # Variable binaria (0, 1) para indicar si el curso i se programa el día d en el periodo p

z = {}  # z[d] = 1 si se utiliza el día d
for d in D: # Días
    z[d] = solver.BoolVar(f'z[{d}]') # Variable binaria (0, 1) para indicar si se utiliza el día d 

#Chat: y = solver.IntVar(0, solver.infinity(), 'y')  # Número mínimo de días necesarios

In [64]:
# Definir y como la suma de las variables z (días utilizados)
y_vars = [solver.BoolVar(f'y[{d}]') for d in D]

# Restricciones de incompatibilidad
y = solver.Sum(y_vars)

### **Función Objetivo**



El objetivo es minimizar el número mínimo de días necesarios para programar los exámenes, representado por la variable **$y$**:

```math
\min y
```

In [65]:
# Función objetivo: minimizar y
solver.Minimize(y)

### **Restricciones**



De manera similar, planteamos nuestras restricciones: se tienen que cumplir las demandas exactas de cada ciudad, sólo se puede transportar a lo máximo la demanda total y la cantidad transportada no puede ser negativa. Todo esto para asegurarnos de que nuestra función objetivo nos dé una solución factible.

- **$y \geq \frac{\sum z_k}{4}$**   (El número mínimo de días está relacionado con los periodos requeridos)

- **$\sum x_{i,j}$**  
  (Se debe garantizar que se asignen los cursos de forma adecuada)

- **$x_{i,k} + x_{j,j} \leq z_k, \quad \forall i \in C$**  
  (Relación entre los cursos asignados y los periodos utilizados)

- **$z_{k+1} \leq z_k, \quad \forall k \in P, \, k = 1, \dots, |P|-1$**  
  (La asignación de periodos debe ser consecutiva)

- **$y \in \mathbb{Z}^+$**  
  (El número mínimo de días debe ser un entero positivo)

- **$x_{i,j} \in \{0,1\}, \quad \forall i \in C, \, j \in P$**  
  (Asignación binaria de cursos a periodos)

- **$z_k \in \{0,1\}, \quad \forall k \in P$**  
  (Uso binario de periodos)


$$
\begin{aligned}
\sum_{d \in D} \sum_{p \in P} x_{i, d, p} & = 1 \quad \forall i \in C \\
x_{i, d, p} + x_{j, d, p} & \leq 1 \quad \forall (i, j) \in I, \, \forall d \in D, \, \forall p \in P\\
\sum_{i \in C} \sum_{p \in P} x_{i, d, p} & \leq |C| \cdot z_d \quad \forall d \in D \\
y & \geq \sum_{d \in D} z_d
\end{aligned}
$$

In [66]:
# Restricción 1: Cada curso debe programarse exactamente una vez
for i in C: # Cursos
    solver.Add(sum(x[i, d, p] for d in D for p in P) == 1) # Cada curso debe programarse exactamente una vez en un día y periodo

# Restricción 2: Cursos incompatibles no pueden programarse al mismo tiempo
for (i, j) in I: # Cursos incompatibles
    for d in D: # Días
        for p in P: # Periodos
            solver.Add(x[i, d, p] + x[j, d, p] <= 1) # Cursos incompatibles no pueden programarse al mismo tiempo

# Restricción 3: Si un periodo de un día se usa, z[d] = 1
for d in D: # Días 
    solver.Add(sum(x[i, d, p] for i in C for p in P) <= len(C) * z[d]) # Si un periodo de un día se usa, z[d] = 1 

# Restricción 4: Relación entre y y los días utilizados
solver.Add(y >= sum(z[d] for d in D)) # Número mínimo de días necesarios 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001AEE4B77780> >

In [67]:
# Resolviendo el problema
solver.Solve() 

0

In [68]:
# Resultados
print("Solución Óptima:")
print(f'Número mínimo de días: {solver.Objective().Value()}')

print("\nAsignación de exámenes:")
for i in C: # Cursos
    for d in D: # Días
        for p in P: # Periodos
            if x[i, d, p].solution_value() == 1: # Si el curso i se programa el día d en el periodo p
                print(f"Curso {i} en día {d} en periodo {p}")
# print("Días utilizados:")
# for d in D:
    # if z[d].solution_value() > 0.5:
        # print(f"  Día {d}")

Solución Óptima:
Número mínimo de días: 2.0

Asignación de exámenes:
Curso 1 en día 1 en periodo 1
Curso 2 en día 1 en periodo 2
Curso 3 en día 1 en periodo 1
Curso 4 en día 1 en periodo 2
Curso 5 en día 1 en periodo 3
Curso 6 en día 1 en periodo 4
Curso 7 en día 2 en periodo 1
Curso 8 en día 1 en periodo 1
Curso 9 en día 1 en periodo 2
Curso 10 en día 2 en periodo 2
Curso 11 en día 2 en periodo 3


Aquí tienes la conversión del documento en LaTeX a Markdown:

---

## Propuesta de una Solución a un Problema de Optimización

El departamento de una universidad debe hacer la programación de los exámenes finales de los cursos que imparte dicho departamento. Cada examen tiene una duración de dos horas. Se necesita determinar el número de días necesario para la realización de dichos exámenes en los siguientes periodos de tiempo: 8:00 a 10:00, 10:15 a 12:15, 14:00 a 16:00 y 16:15 a 18:15, que dan como resultado cuatro periodos de dos horas por día, que deben servir para programar los exámenes de los cursos en distintos salones de clase. Para cada uno de los cursos, se proporciona una lista de los cursos que son incompatibles (ver Tabla 1), es decir, cursos que no se pueden programar al mismo tiempo, porque ambos están siendo por cursados por uno o varios de los estudiantes. Se requiere formular el problema para determinar el número mínimo de días necesarios para la realización de los exámenes de los 11 cursos. 

### Conjuntos de Índices

- **C**: Conjunto de cursos, $C = \{1, 2, \dots, 11\}$
- **P**: Conjunto de periodos, $P = \{1, 2, \dots, 11\}$
- **I**: Conjunto de pares $(i, j)$ tales que $i, j \in C$ y están en conflicto.
- Para cada $i \in C$, **$I_i$**: Conjunto de cursos incompatibles con $i$.  
    Ejemplo: $I_1 = \{2, 5, 7, 10, 11\}$

### Parámetros

Los parámetros capturan la información del problema, que en este caso se refiere a los cursos en conflicto. Esto puede representarse de la siguiente manera:

```math
\forall i, j \in C: 
a_{i,j} = 
\begin{cases} 
1 & \text{si } i \text{ y } j \text{ están en conflicto}, \\ 
0 & \text{en otro caso}.
\end{cases}
```

### Variables de Decisión

Las variables de decisión están relacionadas con la respuesta al problema: determinar en qué periodo se programa el examen de cada curso. 

- Para cada $i \in C$ y $j \in P$: 
    ```math
    x_{i,j} = 
    \begin{cases} 
    1 & \text{si el examen del curso } i \text{ se asigna al periodo } j, \\ 
    0 & \text{en otro caso}.
    \end{cases}
    ```

- Sea **$y$**: el número mínimo de días necesarios para programar los exámenes.

- Para cada $k \in P$: 
    ```math
    z_k = 
    \begin{cases} 
    1 & \text{si se utiliza el periodo } k, \\ 
    0 & \text{en otro caso}.
    \end{cases}
    ```

### Función Objetivo

El objetivo es minimizar el número mínimo de días necesarios para programar los exámenes, representado por la variable **$y$**:

```math
\min y
```

### Restricciones

De manera similar, planteamos nuestras restricciones: se tienen que cumplir las demandas exactas de cada ciudad, sólo se puede transportar a lo máximo la demanda total y la cantidad transportada no puede ser negativa. Todo esto para asegurarnos de que nuestra función objetivo nos dé una solución factible.

Las restricciones garantizan que la solución obtenida por la función objetivo sea factible. Estas se formulan de la siguiente manera:

- **$y \geq \frac{\sum z_k}{4}$**  
  (El número mínimo de días está relacionado con los periodos requeridos)

- **$\sum x_{i,j}$**  
  (Se debe garantizar que se asignen los cursos de forma adecuada)

- **$x_{i,k} + x_{j,j} \leq z_k, \quad \forall i \in C$**  
  (Relación entre los cursos asignados y los periodos utilizados)

- **$z_{k+1} \leq z_k, \quad \forall k \in P, \, k = 1, \dots, |P|-1$**  
  (La asignación de periodos debe ser consecutiva)

- **$y \in \mathbb{Z}^+$**  
  (El número mínimo de días debe ser un entero positivo)

- **$x_{i,j} \in \{0,1\}, \quad \forall i \in C, \, j \in P$**  
  (Asignación binaria de cursos a periodos)

- **$z_k \in \{0,1\}, \quad \forall k \in P$**  
  (Uso binario de periodos)

### Uso de Python y el Solver SCIP

En Python, utilizamos el solver `SCIP` para definir y resolver este problema. Para declarar variables binarias, se utiliza el método:

```python
solver.BoolVar(name)
```

--- 

Este es el formato en Markdown de tu documento en LaTeX. He usado notación en línea para las fórmulas matemáticas y bloques de código para las ecuaciones y ejemplos de Python.

In [1]:
from ortools.linear_solver import pywraplp

# Definición de los cursos y periodos
C = range(1, 12)  # 11 cursos
P = range(1, 5)   # 4 periodos

# Conjunto de incompatibilidades entre cursos (ejemplo basado en la tabla mencionada)
incompatibles = {
    1: {2, 5, 7, 10, 11},
    2: {1, 5, 7, 10, 11},
    3: {4, 5, 6, 7, 10, 11},
    4: {3, 5, 6, 7, 9, 10, 11},
    5: {1, 2, 3, 4, 6, 7, 8, 9, 10, 11},
    6: {3, 4, 5, 7, 9, 10, 11},
    7: {1, 2, 3, 4, 5, 6, 8, 9, 10, 11},
    8: {5, 7, 10, 11},
    9: {3, 5, 6, 7, 10, 11},
    10: {1, 2, 3, 4, 5, 6, 7, 8, 9, 11},
    11: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
}



# Crear el solver usando SCIP
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print("Solver no se pudo crear.")
    exit()

# Variables de decisión
x = {}  # x[i, j] = 1 si el examen de i se programa en el periodo j
z = {}  # z[k] = 1 si se utiliza el periodo k

# Crear las variables binarias
for i in C:
    for j in P:
        x[i, j] = solver.BoolVar(f"x_{i}_{j}")  # Variable binaria

for k in P:
    z[k] = solver.BoolVar(f"z_{k}")  # Variable binaria para periodos

# Variable para el número mínimo de días
y = solver.IntVar(0, 100, "y")  # Variable entera para los días

# Función objetivo: Minimizar el número de días
solver.Minimize(y)

# Restricciones

# 1. El número mínimo de días debe estar relacionado con los periodos utilizados
solver.Add(y >= sum(z[k] for k in P) / 4)

# 2. Cada curso debe ser asignado a un único periodo
for i in C:
    solver.Add(sum(x[i, j] for j in P) == 1)

# 3. Si dos cursos están en conflicto, no pueden ser asignados al mismo periodo
for i in C:
    for j in C:
        if i != j and j in incompatibles.get(i, set()):
            for k in P:
                solver.Add(x[i, k] + x[j, k] <= 1)

# 4. Relación entre los cursos asignados y los periodos utilizados
for i in C:
    for k in P:
        solver.Add(x[i, k] <= z[k])

# 5. Los periodos deben ser consecutivos
for k in range(1, len(P)):
    solver.Add(z[k + 1] <= z[k])

# 6. Los valores de las variables son enteros y binarias
solver.Add(y >= 1)  # El número mínimo de días debe ser un número entero positivo
for k in P:
    solver.Add(z[k] >= 0)  # Los valores de z deben ser binarios
for i in C:
    for j in P:
        solver.Add(x[i, j] >= 0)  # Los valores de x deben ser binarios

# Optimizar el modelo
status = solver.Solve()

# Obtener el resultado
if status == pywraplp.Solver.OPTIMAL:
    print("Número mínimo de días: ", y.solution_value())
    for i in C:
        for j in P:
            if x[i, j].solution_value() > 0.5:  # Si el valor de x[i, j] es 1
                print(f"El examen del curso {i} se programa en el periodo {j}")

    # Mostrar la utilización de los periodos
    for k in P:
        if z[k].solution_value() > 0.5:  # Si el valor de z[k] es 1
            print(f"Se utiliza el periodo {k}")
else:
    print("No se encontró una solución óptima.")


No se encontró una solución óptima.
